# AutoML Mortality Prediction (Colab/Kaggle Ready)

Este notebook implementa un flujo de trabajo de AutoML para predecir la mortalidad intrahospitalaria (`mortality_inhospital`).

## Técnicas de Resampling:
- SMOTEENN
- Borderline-SMOTE + TomekLinks
- Borderline-SMOTE

## Modelos:
- Logistic Regression (LR)
- Random Forest (RF)
- XGBoost (XGB)
- Support Vector Machine (SVM)
- Multi-Layer Perceptron (MLP)
- Gradient Boosting Machine (GBM)
- Bootstrap Aggregating (Bagging)

## Métricas:
- Accuracy
- Recall
- AUC-ROC

In [ ]:
# Instalación de librerías necesarias
# En Kaggle/Colab: Ejecuta esta celda para actualizar las versiones compatibles
!pip install -q --upgrade scikit-learn imbalanced-learn

In [ ]:
# Instalación de librerías necesarias
# En Kaggle: Las librerías ya vienen preinstaladas, puedes saltar esta celda
# En Colab: Descomenta la siguiente línea si es necesario
# !pip install -q imbalanced-learn xgboost pandas scikit-learn matplotlib seaborn joblib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, recall_score
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

# Resampling
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline as ImbPipeline

import warnings
warnings.filterwarnings('ignore')

from pathlib import Path

import joblib
import os

In [ ]:
# Carga de datos
# Si estás en Colab, sube el archivo 'cleaned_dataset_20251104_200505.csv'
try:
    df = pd.read_csv('/content/drive/MyDrive/data/cleaned_dataset_20251104_200505.csv')
    print("Dataset cargado localmente.")
except FileNotFoundError:
    try:
        from google.colab import files
        print("Sube el archivo 'cleaned_dataset_20251104_200505.csv':")
        uploaded = files.upload()
        df = pd.read_csv(next(iter(uploaded.keys())))
    except ImportError:
        print("No se encontró el archivo y no estás en Google Colab. Por favor verifica la ruta.")

print(f"Dimensiones del dataset: {df.shape}")
df.head()

In [ ]:
# Preprocesamiento
target = 'mortality_inhospital'
X = df.drop(columns=[target])
y = df[target]

# Imputación de valores faltantes (Mediana)
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Escalado de datos
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns)

print("Distribución de clases original:")
print(y.value_counts(normalize=True))

In [ ]:
# Definición de técnicas de Resampling
# Nota: Para métodos compuestos usamos una lista de tuplas (pasos individuales)
# 'None' usa un paso "passthrough" que no modifica los datos
from imblearn.over_sampling import SMOTE

resampling_methods = {
    'None (Original)': [],  # Sin resampling - datos originales desbalanceados
    'SMOTE': [('smote', SMOTE(random_state=42))],
    'SMOTEENN': [('smoteenn', SMOTEENN(random_state=42))],
    'BorderlineSMOTE': [('borderline', BorderlineSMOTE(random_state=42, kind='borderline-1'))],
    'BorderlineSMOTE+Tomek': [
        ('borderline', BorderlineSMOTE(random_state=42, kind='borderline-1')),
        ('tomek', TomekLinks())
    ]
}

# Definición de Modelos
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'SVM': SVC(random_state=42, probability=True),
    'MLP': MLPClassifier(random_state=42, max_iter=1000),
    'GBM': GradientBoostingClassifier(random_state=42),
    'Bagging': BaggingClassifier(random_state=42)
}

In [ ]:
# Configuración de directorios para guardar modelos y resultados
MODELS_DIR = Path('trained_models')
RESULTS_FILE = 'training_results.csv'

# Crear directorio si no existe
MODELS_DIR.mkdir(exist_ok=True)

def get_model_filename(res_name, model_name):
    """Genera nombre de archivo seguro para el modelo"""
    safe_res = res_name.replace(' ', '_').replace('(', '').replace(')', '').replace('+', '_')
    safe_model = model_name.replace(' ', '_')
    return MODELS_DIR / f"{safe_res}_{safe_model}.joblib"

def load_or_train_model(res_name, res_steps, model_name, model, X, y, cv):
    """Carga modelo si existe, sino lo entrena y guarda"""
    model_path = get_model_filename(res_name, model_name)
    
    # Verificar si ya existe el modelo entrenado
    if model_path.exists():
        print(f"  - Cargando {model_name} desde cache...")
        saved_data = joblib.load(model_path)
        return saved_data['pipeline'], saved_data['scores']
    
    print(f"  - Entrenando {model_name}...")
    
    # Crear pipeline
    if len(res_steps) == 0:
        from sklearn.pipeline import Pipeline as SkPipeline
        pipeline = SkPipeline([('classifier', model)])
    else:
        pipeline_steps = res_steps + [('classifier', model)]
        pipeline = ImbPipeline(pipeline_steps)
    
    # Calcular métricas con CV
    auc_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='roc_auc')
    recall_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='recall')
    accuracy_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    
    # Entrenar modelo final con todos los datos
    pipeline.fit(X, y)
    
    scores = {
        'auc': auc_scores,
        'recall': recall_scores,
        'accuracy': accuracy_scores
    }
    
    # Guardar modelo y scores
    joblib.dump({'pipeline': pipeline, 'scores': scores}, model_path)
    print(f"    ✓ Modelo guardado en {model_path}")
    
    return pipeline, scores

# Loop de Entrenamiento y Evaluación
results = []
trained_pipelines = {}  # Diccionario para almacenar pipelines entrenados

# Validación Cruzada Estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Verificar si ya existen resultados previos
if os.path.exists(RESULTS_FILE):
    print(f"Encontrado archivo de resultados previos: {RESULTS_FILE}")
    results_df = pd.read_csv(RESULTS_FILE)
    print(f"Cargados {len(results_df)} resultados previos.")
    
    # Cargar modelos existentes
    for res_name in resampling_methods.keys():
        for model_name in models.keys():
            model_path = get_model_filename(res_name, model_name)
            if model_path.exists():
                saved_data = joblib.load(model_path)
                trained_pipelines[f"{res_name}_{model_name}"] = saved_data['pipeline']
else:
    print("Iniciando entrenamiento desde cero...")
    
    for res_name, res_steps in resampling_methods.items():
        print(f"\nEvaluando técnica de resampling: {res_name}")
        
        for model_name, model in models.items():
            try:
                pipeline, scores = load_or_train_model(
                    res_name, res_steps, model_name, model, X_scaled, y, cv
                )
                
                trained_pipelines[f"{res_name}_{model_name}"] = pipeline
                
                results.append({
                    'Resampling': res_name,
                    'Model': model_name,
                    'AUC Mean': scores['auc'].mean(),
                    'AUC Std': scores['auc'].std(),
                    'Recall Mean': scores['recall'].mean(),
                    'Recall Std': scores['recall'].std(),
                    'Accuracy Mean': scores['accuracy'].mean(),
                    'Accuracy Std': scores['accuracy'].std()
                })
            except Exception as e:
                print(f"Error entrenando {model_name} con {res_name}: {e}")
    
    results_df = pd.DataFrame(results)
    
    # Guardar resultados en CSV
    results_df.to_csv(RESULTS_FILE, index=False)
    print(f"\n✓ Resultados guardados en {RESULTS_FILE}")

print(f"\nEntrenamiento completado. {len(trained_pipelines)} modelos disponibles.")

In [ ]:
# Visualización de Resultados
if not results_df.empty:
    results_df_sorted = results_df.sort_values(by='AUC Mean', ascending=False)

    print("Top 10 Modelos por AUC:")
    display(results_df_sorted.head(10))

    # Gráfico de Barras para AUC
    plt.figure(figsize=(14, 8))
    sns.barplot(data=results_df, x='Model', y='AUC Mean', hue='Resampling')
    plt.title('Comparación de AUC por Modelo y Técnica de Resampling')
    plt.xticks(rotation=45)
    plt.ylim(0.5, 1.0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # Gráfico de Barras para Recall
    plt.figure(figsize=(14, 8))
    sns.barplot(data=results_df, x='Model', y='Recall Mean', hue='Resampling')
    plt.title('Comparación de Recall por Modelo y Técnica de Resampling')
    plt.xticks(rotation=45)
    plt.ylim(0.0, 1.0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("No se obtuvieron resultados.")

## Utilidades: Forzar Reentrenamiento

Si necesitas reentrenar todos los modelos desde cero, ejecuta la siguiente celda para eliminar el cache.

In [ ]:
# # ⚠️ EJECUTAR SOLO SI QUIERES REENTRENAR TODO DESDE CERO
# # Esta celda elimina todos los modelos guardados y el archivo de resultados

# FORCE_RETRAIN = False  # Cambiar a True para forzar reentrenamiento

# if FORCE_RETRAIN:
#     import shutil
    
#     # Eliminar directorio de modelos
#     if MODELS_DIR.exists():
#         shutil.rmtree(MODELS_DIR)
#         print(f"✓ Directorio {MODELS_DIR} eliminado")
    
#     # Eliminar archivo de resultados
#     if os.path.exists(RESULTS_FILE):
#         os.remove(RESULTS_FILE)
#         print(f"✓ Archivo {RESULTS_FILE} eliminado")
    
#     print("\n⚠️ Cache eliminado. Vuelve a ejecutar la celda de entrenamiento.")
# else:
#     print("FORCE_RETRAIN está en False. Cambia a True y ejecuta para eliminar el cache.")